# Ingestion & Preprocessing

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

#Viz
import seaborn as sns
import matplotlib.pyplot as plt

#EDA
import xgboost as xgb
import lightgbm as lgb

#from statlearning import plot_dist
#from statlearning import plot_regressions
#from statlearning import plot_coefficients
#from statlearning import plot_feature_importance
#from mpl_toolkits.basemap import Basemap

#from skopt import BayesSearchCV
#from skopt.space import Real, Categorical, Integer

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
#from mlxtend.regressor import StackingCVRegressor

In [2]:
df_train = pd.read_csv('boston_listings.csv',encoding ='ISO-8859-1')

In [3]:
df_train.head()

,id,name,summary,access,interaction,house_rules,host_id,host_since,host_location,host_response_time,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,instant_bookable,is_business_travel_ready,cancellation_policy,reviews_per_month
0,3781,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,Guests solely occupy the 1 floor apartment wit...,We sometimes travel. Always available via: mob...,"No pets, no smoking.",4804,2008/12/3,Massachusetts,within a few hours,...,10.0,10.0,10.0,10,t,NaN,f,f,super_strict_30,0.28
1,5506,**$49 Special ** Private! Minutes to center!,"Private guest room with private bath, You do n...",You get full access to the guest room with pri...,"We give guests privacy, but we are available ...",No Smoking in the Building.,8229,2009/2/19,"Boston, Massachusetts, United States",within an hour,...,10.0,10.0,9.0,10,t,Exempt: This listing is a unit that has contra...,t,f,strict_14_with_grace_period,0.79
2,6695,$99 Special!! Home Away! Condo,"Comfortable, Fully Equipped private apartment...","Full Private apartment. 1 bedroom, kitchen, ...",NaN,No Smoking in the Building.,8229,2009/2/19,"Boston, Massachusetts, United States",within an hour,...,10.0,10.0,9.0,10,t,STR-404620,t,f,strict_14_with_grace_period,0.88
3,8789,Curved Glass Studio/1bd facing Park,"Bright, 1 bed with curved glass windows facing...",Guests have access to the full unit,I'm available for questions and/or issues.,NaN,26988,2009/7/22,"Boston, Massachusetts, United States",within a few hours,...,10.0,10.0,10.0,9,t,NaN,f,f,strict_14_with_grace_period,0.35
4,10730,Bright 1bed facing Golden Dome,"Bright, spacious unit, new galley kitchen, new...",Guests have access to everything in the unit.,I'm available as needed.,"NO SMOKING, NO PETS. $100 move-in fee payable ...",26988,2009/7/22,"Boston, Massachusetts, United States",within a few hours,...,10.0,10.0,10.0,9,t,NaN,f,f,strict_14_with_grace_period,0.24


In [4]:
df_train['price']

0              $125.00 
1              $145.00 
2              $169.00 
3               $99.00 
4              $150.00 
             ...       
3840            $58.00 
3841           $150.00 
3842    Entire home/apt
3843            $50.00 
3844            $68.00 
Name: price, Length: 3845, dtype: object

In [5]:
df_train.isnull().sum()

id                                0
name                              0
summary                          80
access                         1633
interaction                    1252
house_rules                     988
host_id                           2
host_since                        2
host_location                     5
host_response_time              559
host_response_rate              559
host_acceptance_rate            260
host_is_superhost                 1
host_neighbourhood              219
host_total_listings_count         1
host_verifications                1
host_identity_verified            1
neighbourhood                     1
neighbourhood_cleansed            1
zipcode                          14
latitude                          0
longitude                         0
is_location_exact                 0
property_type                     1
room_type                         0
accommodates                      0
bathrooms                         3
bedrooms                    

In [6]:
df_train = df_train.drop(['id','name','summary','access','interaction','house_rules','host_id','zipcode',
                          'requires_license','license','review_scores_rating','review_scores_accuracy',
                          'review_scores_cleanliness','review_scores_checkin','review_scores_communication',
                         'review_scores_location','review_scores_value','reviews_per_month','host_response_time',
                         'host_response_rate','host_acceptance_rate','host_neighbourhood','host_is_superhost','host_neighbourhood'],axis = 1)

In [7]:
df_train.isnull().sum()

host_since                     2
host_location                  5
host_total_listings_count      1
host_verifications             1
host_identity_verified         1
neighbourhood                  1
neighbourhood_cleansed         1
latitude                       0
longitude                      0
is_location_exact              0
property_type                  1
room_type                      0
accommodates                   0
bathrooms                      3
bedrooms                       4
beds                          20
bed_type                       1
amenities_dict                 0
price                          1
cleaning_fee                 396
availability_30                0
availability_60                0
availability_90                0
availability_365               0
number_of_reviews              1
instant_bookable               4
is_business_travel_ready       4
cancellation_policy            3
dtype: int64

In [8]:
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
#pd.set_option('max_colwidth',200)
#pd.set_option('expand_frame_repr', False)
#df_train.head()

In [9]:
filled_train = df_train.copy()

In [10]:
#填补固定数字
filled_train['cleaning_fee'] = filled_train['cleaning_fee'].fillna(0)

filled_train = filled_train.dropna()
print(filled_train.shape)

(3805, 28)


In [11]:
filled_train.isnull().sum()

host_since                   0
host_location                0
host_total_listings_count    0
host_verifications           0
host_identity_verified       0
neighbourhood                0
neighbourhood_cleansed       0
latitude                     0
longitude                    0
is_location_exact            0
property_type                0
room_type                    0
accommodates                 0
bathrooms                    0
bedrooms                     0
beds                         0
bed_type                     0
amenities_dict               0
price                        0
cleaning_fee                 0
availability_30              0
availability_60              0
availability_90              0
availability_365             0
number_of_reviews            0
instant_bookable             0
is_business_travel_ready     0
cancellation_policy          0
dtype: int64

In [12]:
filled_train['price']

0       $125.00 
1       $145.00 
2       $169.00 
3        $99.00 
4       $150.00 
          ...   
3838    $100.00 
3839     $52.00 
3840     $58.00 
3841    $150.00 
3844     $68.00 
Name: price, Length: 3805, dtype: object

In [13]:
DF_train = filled_train.to_csv("DF_train.csv")

In [14]:
DF_train_use = pd.read_csv('DF_train_use.csv',encoding ='ISO-8859-1',index_col = 0)

In [15]:
DF_train_use.head()

,host_since,host_location,host_total_listings_count,host_verifications,host_identity_verified,neighbourhood,neighbourhood_cleansed,latitude,longitude,is_location_exact,...,price,cleaning_fee,availability_30,availability_60,availability_90,availability_365,number_of_reviews,instant_bookable,is_business_travel_ready,cancellation_policy
0,2008/12/3,Massachusetts,5,"['email', 'phone', 'reviews']",f,East Boston,East Boston,42.36413,-71.02991,t,...,125,75,11,20,20,97,16,f,f,super_strict_30
1,2009/2/19,"Boston, Massachusetts, United States",2,"['email', 'phone', 'reviews', 'kba']",t,Roxbury,Roxbury,42.32981,-71.09559,t,...,145,60,1,14,44,307,106,t,f,strict_14_with_grace_period
2,2009/2/19,"Boston, Massachusetts, United States",2,"['email', 'phone', 'reviews', 'kba']",t,Roxbury,Roxbury,42.32994,-71.09351,t,...,169,80,1,4,19,266,115,t,f,strict_14_with_grace_period
3,2009/7/22,"Boston, Massachusetts, United States",10,"['email', 'phone', 'reviews', 'jumio', 'offlin...",f,Beacon Hill,Downtown,42.35919,-71.06265,t,...,99,250,17,42,42,290,24,f,f,strict_14_with_grace_period
4,2009/7/22,"Boston, Massachusetts, United States",10,"['email', 'phone', 'reviews', 'jumio', 'offlin...",f,Beacon Hill,Downtown,42.35840,-71.06185,t,...,150,250,28,29,29,42,31,f,f,strict_14_with_grace_period


In [16]:
print(DF_train_use['host_identity_verified'].value_counts())
print(' ')
print(DF_train_use['is_location_exact'].value_counts())
print(' ')
print(DF_train_use['cancellation_policy'].value_counts())
print(' ')
print(DF_train_use['instant_bookable'].value_counts())
print(' ')
print(DF_train_use['is_business_travel_ready'].value_counts())
print(' ')

f    2473
t    1306
Name: host_identity_verified, dtype: int64
 
t    3578
f     201
Name: is_location_exact, dtype: int64
 
strict_14_with_grace_period    1914
moderate                        884
flexible                        822
strict                          116
super_strict_30                  36
super_strict_60                   7
Name: cancellation_policy, dtype: int64
 
t    1925
f    1854
Name: instant_bookable, dtype: int64
 
f    3779
Name: is_business_travel_ready, dtype: int64
 


In [17]:
#将有顺序关系的变量直接替换为数字
ordinal_variable = ['host_identity_verified','is_location_exact','instant_bookable','is_business_travel_ready','cancellation_policy']
DF_train_use[ordinal_variable] = DF_train_use[ordinal_variable].replace({
    'flexible':6,'moderate':4,'strict':3,'super_strict_30':2,'super_strict_60':1,'t':1,'f':0})


In [18]:
DF_train_dm = pd.get_dummies(DF_train_use, drop_first=True).copy()

In [19]:
DF_train_dm

,host_total_listings_count,host_identity_verified,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,beds,price,...,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_2,cancellation_policy_3,cancellation_policy_4,cancellation_policy_6,cancellation_policy_strict_14_with_grace_period
0,5,0,42.364130,-71.02991,1,2,1.0,1,0,125,...,0,0,0,0,1,1,0,0,0,0
1,2,1,42.329810,-71.09559,1,2,1.0,1,1,145,...,0,0,0,0,1,0,0,0,0,1
2,2,1,42.329940,-71.09351,1,4,1.0,1,1,169,...,0,0,0,0,1,0,0,0,0,1
3,10,0,42.359190,-71.06265,1,2,1.0,1,1,99,...,0,0,0,0,1,0,0,0,0,1
4,10,0,42.358400,-71.06185,1,2,1.0,1,1,150,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3838,11,0,42.325370,-71.10501,0,1,2.0,0,1,100,...,0,0,0,0,1,0,0,0,0,1
3839,1,0,42.363920,-71.05598,1,1,1.0,1,1,52,...,0,0,0,0,1,0,0,0,0,1
3840,1,0,42.364150,-71.05685,1,1,1.0,1,1,58,...,0,0,0,0,1,0,0,0,1,0
3841,10,0,42.373640,-71.03714,1,4,1.0,2,2,150,...,0,0,0,0,1,0,0,0,1,0


In [20]:
DF_train_dm.to_csv("DF_train_dm.csv")

# 统计分析

In [21]:
correlation = DF_train_dm.corr()['price'].sort_values()
correlation_gd = correlation.loc[(correlation > 0.1)|(correlation < -0.1)]

In [22]:
DF_train_dm = DF_train_dm[correlation_gd.index].copy()
print(DF_train_dm.shape)

(3779, 13)


In [23]:
continuous = DF_train_dm.iloc[:,:25].columns

In [24]:
for i in continuous:
    DF_train_dm[i] = np.log1p(DF_train_dm[i])

In [25]:
DF_train_log_mm = DF_train_dm.copy()
for i in continuous:
    DF_train_log_mm[i] = (DF_train_log_mm[i] - DF_train_log_mm[i].min())/(DF_train_log_mm[i].max() - DF_train_log_mm[i].min())

DF_train_log_mm.head()

,cleaning_fee,host_since_2019/6/28,host_since_2019/10/8,neighbourhood_Leather District,bedrooms,property_type_Hotel,"host_verifications_['email', 'phone']",beds,accommodates,property_type_Boutique hotel,host_since_2019/10/29,host_since_2019/6/5,price
0,0.696639,0.0,0.0,0.0,0.26265,0.0,0.0,0.000000,0.149726,0.0,0.0,0.0,0.296160
1,0.661273,0.0,0.0,0.0,0.26265,0.0,0.0,0.221065,0.149726,0.0,0.0,0.0,0.319866
2,0.706889,0.0,0.0,0.0,0.26265,0.0,0.0,0.221065,0.338358,0.0,0.0,0.0,0.344355
3,0.888821,0.0,0.0,0.0,0.26265,0.0,0.0,0.221065,0.149726,0.0,0.0,0.0,0.258972
4,0.888821,0.0,0.0,0.0,0.26265,0.0,0.0,0.221065,0.149726,0.0,0.0,0.0,0.325284


In [26]:
DF_train_log_std = DF_train_dm.copy()
for i in range(DF_train_log_std.shape[1]):
    DF_train_log_std.iloc[:,i] = (DF_train_log_std.iloc[:,i] - DF_train_log_std.iloc[:,i].mean()) / DF_train_log_std.iloc[:,i].std()
DF_train_log_std.head()

,cleaning_fee,host_since_2019/6/28,host_since_2019/10/8,neighbourhood_Leather District,bedrooms,property_type_Hotel,"host_verifications_['email', 'phone']",beds,accommodates,property_type_Boutique hotel,host_since_2019/10/29,host_since_2019/6/5,price
0,0.485540,-0.058777,-0.048859,-0.016266,-0.174829,-0.115784,-0.253418,-2.416577,-0.555018,-0.069214,-0.02301,-0.02301,-0.076636
1,0.362097,-0.058777,-0.048859,-0.016266,-0.174829,-0.115784,-0.253418,-0.604417,-0.555018,-0.069214,-0.02301,-0.02301,0.132923
2,0.521314,-0.058777,-0.048859,-0.016266,-0.174829,-0.115784,-0.253418,-0.604417,0.618948,-0.069214,-0.02301,-0.02301,0.349404
3,1.156328,-0.058777,-0.048859,-0.016266,-0.174829,-0.115784,-0.253418,-0.604417,-0.555018,-0.069214,-0.02301,-0.02301,-0.405375
4,1.156328,-0.058777,-0.048859,-0.016266,-0.174829,-0.115784,-0.253418,-0.604417,-0.555018,-0.069214,-0.02301,-0.02301,0.180820


In [27]:
from sklearn.model_selection import train_test_split

X_train = DF_train_dm.drop('price',axis = 1)
y_train = DF_train_dm['price']

X_train, X_test, y_train, y_test = train_test_split( X_train, y_train, test_size=0.2, random_state=42)

In [28]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression(fit_intercept = True)
LR_model.fit(X_train, y_train )

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
LR_model.coef_

array([ 0.05036911,  0.90587542,  2.34928065,  4.54979825, -0.15810356,
        0.88837014,  0.07030487, -0.14722207,  1.08831533,  0.60266551,
        5.78786852,  6.19637544])

In [30]:
prediction = np.exp(LR_model.predict(X_test))-1
prediction

array([ 164.26409507,  295.36187412,  164.19071394,   66.29289968,
        109.74872518,  116.31044592,  175.20538024,   69.84180954,
        171.61469479,  196.09928611,  125.21295618,  170.46779904,
         69.62840076,   96.45671309,  121.89278252,  183.41836583,
        227.29683459,   62.87114547,   87.00219429,   65.43098524,
        143.54743108,  135.91599773,  103.61936025,   67.24725195,
        488.99161406,  107.87666303,   87.00219429,  118.04525212,
         98.30212704,  116.31044592,  121.12998931,  198.94291897,
         68.00164924,  121.89278252,   55.6044642 ,  224.81054548,
         87.00219429,  121.52257177,  188.22775723,  204.18582736,
        192.09904758,  111.27609332,  198.94291897,  141.34429874,
        116.31044592,  165.45865862,  100.19148077,  226.45186952,
        141.98153044,   87.00219429,  213.80038032,  192.59350271,
        264.60021258,   68.00164924,  126.79209865,  143.16366457,
        215.73192601,   87.00219429,  198.94291897,  125.21295

In [31]:
import statsmodels.api as sm

LR_sm_model = sm.OLS(y_train,sm.add_constant(X_train)).fit()
print(LR_sm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.484
Model:                            OLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                     235.4
Date:                Thu, 17 Sep 2020   Prob (F-statistic):               0.00
Time:                        19:42:52   Log-Likelihood:                -2227.5
No. Observations:                3023   AIC:                             4481.
Df Residuals:                    3010   BIC:                             4559.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [32]:
np.exp(LR_sm_model.predict(sm.add_constant(X_test)))-1

1304    164.264095
2647    295.361874
2383    164.190714
301      66.292900
1134    109.748725
           ...    
111     181.375400
3101    109.748725
3700     87.002194
2952    214.537935
2853    158.845883
Length: 756, dtype: float64

# FE_dm

In [33]:
df = pd.read_csv('DF_train_dm.csv',encoding ='ISO-8859-1',index_col = 0)

In [37]:
FE_dm_train, FE_dm_test = train_test_split(df,test_size=0.2, random_state=42)

In [38]:
FE_dm_train

,host_total_listings_count,host_identity_verified,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,beds,price,...,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_2,cancellation_policy_3,cancellation_policy_4,cancellation_policy_6,cancellation_policy_strict_14_with_grace_period
3595,1451,0,42.35120,-71.06519,1,2,1.0,0,1,171,...,0,0,0,0,1,0,0,0,1,0
547,89,0,42.36017,-71.06346,1,2,1.0,0,1,150,...,0,0,0,0,1,0,0,0,1,0
2387,5,0,42.34998,-71.07855,1,2,1.0,1,1,113,...,0,0,0,0,1,0,0,0,1,0
3357,1451,0,42.36738,-71.03648,1,2,1.0,1,1,102,...,0,0,0,0,1,0,0,0,1,0
1630,1,0,42.31575,-71.11526,1,4,1.0,2,2,120,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141,8,1,42.37497,-71.03751,1,2,2.0,1,1,49,...,0,0,0,0,1,0,0,0,0,1
1310,686,0,42.32897,-71.11107,1,5,2.0,2,1,170,...,0,0,0,0,1,0,0,0,0,1
866,22,0,42.32457,-71.05838,1,1,1.0,1,1,75,...,0,0,0,0,1,0,0,1,0,0
3555,34,0,42.35555,-71.06281,1,2,1.0,1,1,175,...,0,0,0,0,1,0,0,1,0,0


In [39]:
FE_dm_test

,host_total_listings_count,host_identity_verified,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,beds,price,...,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_2,cancellation_policy_3,cancellation_policy_4,cancellation_policy_6,cancellation_policy_strict_14_with_grace_period
1304,1,1,42.28829,-71.12869,0,4,2.0,2,2,170,...,0,0,0,0,1,0,0,1,0,0
2647,4,0,42.31680,-71.05842,1,8,2.0,2,6,500,...,0,0,0,0,1,0,0,1,0,0
2383,57,0,42.36013,-71.05358,1,3,1.0,0,1,155,...,0,0,0,0,1,0,0,1,0,0
301,2,1,42.31261,-71.06710,1,1,2.0,1,1,45,...,0,0,0,0,1,0,0,0,0,1
1134,834,0,42.34174,-71.04810,1,2,1.0,1,1,90,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,3,1,42.34840,-71.15696,1,4,1.0,1,2,140,...,0,0,0,0,1,0,0,0,0,1
3101,39,0,42.35873,-71.06337,1,2,1.0,1,1,157,...,0,0,0,0,1,0,0,0,0,1
3700,1,0,42.33929,-71.04978,1,2,1.0,1,1,250,...,0,0,0,0,1,0,0,0,1,0
2952,0,0,42.30815,-71.08586,1,6,1.0,3,4,150,...,0,0,0,0,1,0,0,0,0,1


# EDA